In [ ]:
import torch
from matplotlib import pyplot as plt
from torchvision.utils import make_grid
from torch_tools.visualization import to_image
from visualization import interpolate
from loading import load_from_dir
%matplotlib inline

rows = 8
plt.figure(figsize=(5, rows), dpi=150)


deformator, G, shift_predictor = load_from_dir('./models/pretrained/deformators/BigGAN/')
discovered_annotation = ''
for d in deformator.directions_dict.items():
    discovered_annotation += '{}: {}\n'.format(d[0], d[1])
print('Interpretable directions:\n' + discovered_annotation)

inspection_dim = list(deformator.directions_dict.values())[0]


zs = torch.randn([rows, G.dim_z] if type(G.dim_z) == int else [rows] + G.dim_z, device='cuda')


for z, i in zip(zs, range(rows)):
    interpolation_deformed = interpolate(
        G, z.unsqueeze(0),
        shifts_r=7,
        shifts_count=3,
        dim=inspection_dim,
        deformator=deformator,
        with_central_border=True)

    plt.subplot(rows, 1, i + 1)
    plt.axis('off')
    grid = make_grid(interpolation_deformed, nrow=11, padding=1, pad_value=0.0)
    grid = torch.clamp(grid, -1, 1)

    plt.imshow(to_image(grid))